# Anomalous sound detection: Machine Learning with wavelet & Gradient Boosting

In [1]:
#Let's first import the module that we will need
%matplotlib inline
import librosa
import librosa.display
import IPython.display as ipd
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np

In [2]:
#Constant used throughout the notebook
PATH_TRAINING_DATASET = r"C:\Users\sylv_\Documents\audio for asd"

In [3]:
def build_dataframe(machine_str = 'valve'):
    #Get list of files in train and test directory
    path_train_folder = PATH_TRAINING_DATASET + "\\dev_" + machine_str + "\\" + machine_str + "\\train"
    path_test_folder = PATH_TRAINING_DATASET  + "\\dev_" + machine_str + "\\" + machine_str + "\\test"

    train_files = [f for f in os.listdir(path_train_folder)]
    test_files = [f for f in os.listdir(path_test_folder)]

    #Get list of dictionnary for creating DataFrame
    list_dict_file = []

    #Loop through filenames
    for filename in train_files:

        #Get filename as list of string
        splitted_filename = filename.split('_')

        #Append dictionnary to list
        list_dict_file.append({
            'filepath' : path_train_folder + "\\" + filename,
            'filename' : filename,
            'section' : int(splitted_filename[1]),
            'domain_env' : splitted_filename[2],
            'dir' : splitted_filename[3],
            'sound_type' : splitted_filename[4],
            'id' : splitted_filename[5],
            'suffix' : '_'.join(splitted_filename[6:])        #.split('.')[0]  --> want to keep the complete suffix
        })

    #Loop through filenames
    for filename in test_files:

        #Get filename as list of string
        splitted_filename = filename.split('_')

        #Append dictionnary to list
        list_dict_file.append({
            'filepath' : path_test_folder  + "\\" + filename,
            'filename' : filename,
            'section' : int(splitted_filename[1]),
            'domain_env' : splitted_filename[2],
            'dir' : splitted_filename[3],
            'sound_type' : splitted_filename[4],
            'id' : splitted_filename[5],
            'suffix' : '_'.join(splitted_filename[6:])     #.split('.')[0]  --> want to keep the complete suffix
        })
        
    return pd.DataFrame(list_dict_file)


def sound_from_file(filename, machine_str = 'valve', dataset = 'train'):
    df = build_dataframe(machine_str)
    path = PATH_TRAINING_DATASET + "\\dev_" + machine_str + "\\" + machine_str + "\\" + dataset + "\\" + filename
    return df[df['filepath']==path].iloc[0]
    


df_fan = build_dataframe('fan')
df_valve = build_dataframe('valve')
df_bearing = build_dataframe('bearing')
df_slider = build_dataframe('slider')
df_car = build_dataframe('ToyCar')
df_train = build_dataframe('ToyTrain')
df_gearbox = build_dataframe('gearbox')

df_fan.info()
df_fan.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filepath    3600 non-null   object
 1   filename    3600 non-null   object
 2   section     3600 non-null   int64 
 3   domain_env  3600 non-null   object
 4   dir         3600 non-null   object
 5   sound_type  3600 non-null   object
 6   id          3600 non-null   object
 7   suffix      3600 non-null   object
dtypes: int64(1), object(7)
memory usage: 225.1+ KB


,filepath,filename,section,domain_env,dir,sound_type,id,suffix
0,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_00_source_train_normal_0000_m-n_W.wav,0,source,train,normal,0000,m-n_W.wav
1,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_00_source_train_normal_0001_m-n_X.wav,0,source,train,normal,0001,m-n_X.wav
2,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_00_source_train_normal_0002_m-n_X.wav,0,source,train,normal,0002,m-n_X.wav
3,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_00_source_train_normal_0003_m-n_W.wav,0,source,train,normal,0003,m-n_W.wav
4,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_00_source_train_normal_0004_m-n_W.wav,0,source,train,normal,0004,m-n_W.wav


### Other modules and definitions

In [4]:
import pywt
print(pywt.families(short=False))
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
import scipy
from sklearn.metrics import roc_auc_score
pywt.wavelist('db')

['Haar', 'Daubechies', 'Symlets', 'Coiflets', 'Biorthogonal', 'Reverse biorthogonal', 'Discrete Meyer (FIR Approximation)', 'Gaussian', 'Mexican hat wavelet', 'Morlet wavelet', 'Complex Gaussian wavelets', 'Shannon wavelets', 'Frequency B-Spline wavelets', 'Complex Morlet wavelets']


['db1',
 'db2',
 'db3',
 'db4',
 'db5',
 'db6',
 'db7',
 'db8',
 'db9',
 'db10',
 'db11',
 'db12',
 'db13',
 'db14',
 'db15',
 'db16',
 'db17',
 'db18',
 'db19',
 'db20',
 'db21',
 'db22',
 'db23',
 'db24',
 'db25',
 'db26',
 'db27',
 'db28',
 'db29',
 'db30',
 'db31',
 'db32',
 'db33',
 'db34',
 'db35',
 'db36',
 'db37',
 'db38']

In [5]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]
 
def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]
 
def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

def get_audio_features(df, waveletname):
    list_features = []
    for audio_file in df['filepath']:
        signal=librosa.load(audio_file, sr=None)[0]
        list_coeff = pywt.wavedec(signal, waveletname)
        features = []
        for coeff in list_coeff:
            features += get_features(coeff)
        list_features.append(features)
    return list_features

### Data selection

In [6]:
df0 = df_fan # to change for other machines

df1=df0[df0['sound_type']=='normal'].sample(n=500).reset_index()
df2=df0[df0['sound_type']=='anomaly'].sample(n=300).reset_index()
df_use0= pd.concat([df1,df2]).reset_index()
df_use0= df_use0.drop('level_0', axis = 1)
df_use0= df_use0.drop('index', axis = 1)
target1= df_use0['sound_type']
target = np.where(target1=='normal',1,-1)

df_use0.tail()

,filepath,filename,section,domain_env,dir,sound_type,id,suffix
795,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_01_source_test_anomaly_0012_f-n_A.wav,1,source,test,anomaly,0012,f-n_A.wav
796,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_01_target_test_anomaly_0040_f-n_C.wav,1,target,test,anomaly,0040,f-n_C.wav
797,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_00_source_test_anomaly_0015_m-n_W.wav,0,source,test,anomaly,0015,m-n_W.wav
798,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_01_target_test_anomaly_0029_f-n_C.wav,1,target,test,anomaly,0029,f-n_C.wav
799,C:\Users\sylv_\Documents\audio for asd\dev_fan...,section_02_target_test_anomaly_0036_n-lv_L4.wav,2,target,test,anomaly,0036,n-lv_L4.wav


### Audio loading &Training 

In [7]:
X_train,X_test,y_train,y_test = train_test_split(get_audio_features(df_use0, 'db8'), 
                                                 target,test_size=0.2) # change 'db8' for other type of wavelets

cls = GradientBoostingClassifier(n_estimators=2000) # Try other classification
cls.fit(X_train, y_train)

## this cell can take some time when executed (load et train)

GradientBoostingClassifier(n_estimators=2000)

### Score

In [8]:
train_score = cls.score(X_train, y_train)
test_score = cls.score(X_test, y_test)
print("Train Score for the dataset is about: {}".format(train_score))
print("Test Score for the dataset is about: {}".format(test_score))
preds= cls.predict(X_test)
roc_auc_score(y_test, preds)

Train Score for the dataset is about: 1.0
Test Score for the dataset is about: 0.8375


0.827786057294254